In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF']='max_split_size_mb:128'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from ..src.training.training_utils import *

import transformers

transformers.logging.set_verbosity_debug()

/workspace/conda/envs/aidx-model-development/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def main():
    parquet_dir = '/workspace/data'
    stream_data = False
    base_model_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
    
    cpu_count = os.cpu_count()
    gpu_count = torch.cuda.device_count()
    
    print("Setting up training environment...")
    accelerator = setup_training_env()
    data_collator = create_data_collator(base_model_id)
    accelerator.print("Done setting up training environment...")
    
    accelerator.print(f"Number of CPUs available: {cpu_count}")
    accelerator.print(f"Number of GPUs available: {gpu_count}")
    
    accelerator.print("Loading data...")
    train_data, val_data, train_count, val_count = load_data(parquet_dir, stream=stream_data, cpu_count=cpu_count, accelerator=accelerator)
    accelerator.print("Done loading data...")
    
    accelerator.print("Loading compute_bertscore...")
    # compute_bertscore = load_bertscore()
    accelerator.print("Done loading compute_bertscore...")

    # x = train_data.select(range(100))
    
    # def test(a):
    #     return {
    #         'attention_mask': a['attention_mask'][:512],
    #         'input_ids': a['input_ids'][:512]
    #     }
    
    # x = x.map(test)
    
    accelerator.print("Loading model trainer...")
    trainer = load_model_trainer(
            base_model_id,
            False,
            data_collator,
            train_data,
            val_data,
            train_count,
            cpu_count//gpu_count,
            accelerator
        )
    
    print_gpu_utilization(accelerator)

    train_result = trainer.train()

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(main, num_processes=2)

Launching training on 2 GPUs.
Setting up training environment...
Setting up training environment...


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Currently logged in as: alan-alwakeel (aidx). Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: alan-alwakeel (aidx). Use `wandb login --relogin` to force relogin


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00012-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00013-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00014-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00015-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00016-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]